<a href="https://colab.research.google.com/github/Metaboll/Nano-Robi/blob/master/Inferencias_Espanol3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# 8/4/2020
# Version de Spacy es  2.2.4 (anterior 2019 ---> spacy==2.0.18)

# things we need for NLP
#https://towardsdatascience.com/build-it-yourself-chatbot-api-with-keras-tensorflow-model-f6d75ce957a5
# json EDITOR

# EJEMPLOS SPACY
#https://spacy.io/usage/examples

# tree para spacy
#https://stackoverflow.com/questions/39323325/can-i-find-subject-from-spacy-dependency-tree-using-nltk-in-python


#
#https://jsoneditoronline.org/
#
#spacy==2.0.18
#https://github.com/explosion/spaCy/issues/3451

# ***************** CUANDO TENIA PROBLEMAS CON LA RASPBERRY Y SPACY
#!pip install -U spacy==2.0.18
#!python -m spacy download en_core_web_sm

# EN ESPAÑOL
!pip install -U spacy
print("***"*10)
!python -m spacy download es_core_news_sm
!python -m nltk.downloader all

from IPython.display import clear_output
clear_output()
import spacy
print(" Version de Spacy es ", spacy.__version__)

 Version de Spacy es  2.2.4


     |████████████████████████████████| 1.4MB 4.7MB/s 
  Created wheel for nltk: filename=nltk-3.5-cp36-none-any.whl size=1434671 sha256=7ee9ad8ef8e2ec21a6eca0eb70130711dd7d5e6583ca41c92110d6a05f88a698
  Stored in directory: /root/.cache/pip/wheels/ae/8c/3f/b1fe0ba04555b08b57ab52ab7f86023639a526d8bc8d384306
Successfully built nltk
  Found existing installation: nltk 3.2.5
    Uninstalling nltk-3.2.5:
      Successfully uninstalled nltk-3.2.5


In [0]:
#INSTRUCCIONES

!pip uninstall PyYAML -y
!pip install chatterbot
!pip install chatterbot_corpus

In [0]:
import nltk
from nltk.stem.lancaster import LancasterStemmer


stemmer = LancasterStemmer()

# things we need for Tensorflow
import numpy as np

import pandas as pd
import pickle
import random

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
import os
import json


with open('drive/My Drive/data/data/intents_generales.json') as json_data: #intents_generales.json // intents_preguntas.json //intents_generales #intents_spanish_1
    intents = json.load(json_data)

print("Everything seems Ok")

Everything seems Ok


In [0]:
words = []
classes = []
documents = []
ignore_words = ['?']
# loop through each sentence in our intents patterns
for intent in intents['intents']:
    for pattern in intent['patterns']:
        # tokenize each word in the sentence
        w = nltk.word_tokenize(pattern)
        # add to our words list
        words.extend(w)
        # add to documents in our corpus
        documents.append((w, intent['tag']))
        # add to our classes list
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

# stem and lower each word and remove duplicates
words = [stemmer.stem(w.lower()) for w in words if w not in ignore_words]
words = sorted(list(set(words)))

# sort classes
classes = sorted(list(set(classes)))

# documents = combination between patterns and intents
print (len(documents), "documents")
print(documents[0])
print(documents[100])
# classes = intents
print (len(classes), "classes", classes)
# words = all words, vocabulary
print (len(words), "unique stemmed words", words)

322 documents
(['Hola'], 'Saludos')
(['La', 'moto', 'es', 'muy', 'rapida'], 'Ser')
11 classes ['Adios', 'Duda', 'Esta', 'Gracias', 'Insultos', 'Peticionayuda', 'Preguntas', 'Saludos', 'Ser', 'matematicas', 'orden']
444 unique stemmed words ['1', '10', '100', '1000', '2', '3', '4', '5', '6', '68', '7+8', 'a', 'abajo', 'abr', 'acero', 'adio', 'ag', 'agil', 'aguil', 'al', 'alt', 'altisimo', 'alto', 'amarillo', 'amorfo', 'an', 'anillo', 'apag', 'aplicac', 'aplicación', 'armando', 'armario', 'arreglado', 'av', 'avisp', 'ayud', 'ayudarm', 'ayudarno', 'azuc', 'año', 'bajito', 'bajo', 'barato', 'barco', 'basur', 'bien', 'bitch', 'blanca', 'blanco', 'bonit', 'bonito', 'borr', 'bril', 'buena', 'bueno', 'cabron', 'cal', 'cam', 'capit', 'caro', 'carrer', 'cas', 'chin', 'ciento', 'cinco', 'ciudad', 'claud', 'coch', 'cocin', 'colegio', 'comid', 'como', 'comodo', 'compar', 'compr', 'contestart', 'continu', 'cop', 'cordob', 'corrupto', 'cort', 'cos', 'coseno', 'criad', 'cuadrad', 'cual', 'cuando', 'cu

In [0]:
# create our training data
training = []
# create an empty array for our output
output_empty = [0] * len(classes)

# training set, bag of words for each sentence
for doc in documents:
    # initialize our bag of words
    bag = []
    # list of tokenized words for the pattern
    pattern_words = doc[0]
    # stem each word - create base word, in attempt to represent related words
    pattern_words = [stemmer.stem(word.lower()) for word in pattern_words]
    # create our bag of words array with 1, if word match found in current pattern
    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)
    
    # output is a '0' for each tag and '1' for current tag (for each pattern)
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1
    
    training.append([bag, output_row])

# shuffle our features and turn into np.array
random.shuffle(training)
training = np.array(training)

# create train and test lists. X - patterns, Y - intents
train_x = list(training[:,0])
train_y = list(training[:,1])

In [0]:
#!pip install -U keras
!pip install keras==2.3.1
#from keras.models import Sequential
#from keras.layers import Dense, Activation, Dropout
#from keras.optimizers import SGD
from tensorflow.python.keras.layers import Dense, Activation, Dropout
from tensorflow.python.keras import Sequential
from tensorflow.keras.optimizers import SGD


In [0]:
# Create model - 3 layers. First layer 128 neurons, second layer 64 neurons and 3rd output layer contains number of neurons
# equal to number of intents to predict output intent with softmax
model = Sequential()
model.add(Dense(128, input_shape=(len(train_x[0]),), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))

model.add(Dense(len(train_y[0]), activation='softmax'))
print("It's seems correctly")

It's seem all correctly


In [0]:
# Compile model. Stochastic gradient descent with Nesterov accelerated gradient gives good results for this model
sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)

model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])
print("paso")

paso


In [0]:
# Fit the model
model.fit(np.array(train_x), np.array(train_y), epochs=200, batch_size=5, verbose=1)

In [0]:
def clean_up_sentence(sentence):
    # tokenize the pattern - split words into array
    sentence_words = nltk.word_tokenize(sentence)
    # stem each word - create short form for word
    sentence_words = [stemmer.stem(word.lower()) for word in sentence_words]
    return sentence_words

# return bag of words array: 0 or 1 for each word in the bag that exists in the sentence
def bow(sentence, words, show_details=True):
    # tokenize the pattern
    sentence_words = clean_up_sentence(sentence)
    # bag of words - matrix of N words, vocabulary matrix
    bag = [0]*len(words)  
    for s in sentence_words:
        for i,w in enumerate(words):
            if w == s: 
                # assign 1 if current word is in the vocabulary position
                bag[i] = 1
                if show_details:
                    print ("found in bag: %s" % w)

    return(np.array(bag))

In [0]:
p = bow("Adios", words)
print (p)
print (classes)

found in bag: adio
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
['Adios', 'Duda', 'Esta', 'Gracias', 'Insultos', 'Peticionayuda', 'Preguntas', '

In [0]:

p = bow("Ella fue muy alta", words)
#You are boring da problems
#print (p)
#print (classes)
inputvar = pd.DataFrame([p], dtype=float, index=['input'])
results = model.predict(inputvar)
results_index = np.argmax(results)
print(" El numero de las etiquetas  que es solucion es ",  results_index)
print(results[0][results_index])
results[0][results_index] = 0
tag = classes[results_index]
print(tag)



 El numero de las etiquetas  que es solucion es  2
0.99950516
Siono


In [0]:
#Probelma con ; Como te llamas
while True:
    message=input('You: ')
    if message.strip() != 'Bye':
        p = bow(message, words)
        inputvar = pd.DataFrame([p], dtype=float, index=['input'])
        results = model.predict(inputvar)
        results_index = np.argmax(results)
        #print(" El numero de las etiquetas  que es solucion es ",  results_index)
        print("Confianza del resultado >>>>>>" ,results[0][results_index])
        results[0][results_index] = 0
        tag = classes[results_index]
        print(" El ambito de la conversacio´n es ; ", tag)
        
        
    if message.strip() == 'Bye':
        print('ChatBot : Bye')
        break

You: Como te llamas
found in bag: como
found in bag: te
found in bag: llama
Confianza del resultado >>>>>> 1.0
 El ambito de la conversacio´n es ;  quieneres
You: que nombre tienes
found in bag: que
found in bag: nombr
found in bag: tien
Confianza del resultado >>>>>> 0.9999889
 El ambito de la conversacio´n es ;  quieneres
You: Cuanto pesas
found in bag: cuanto
found in bag: pesa
Confianza del resultado >>>>>> 0.99995565
 El ambito de la conversacio´n es ;  cuantopesa
You: Que mides
found in bag: que
found in bag: mid
Confianza del resultado >>>>>> 0.7478779
 El ambito de la conversacio´n es ;  quequieres
You: Cuanta distancia hay entre malaga y sevilla
found in bag: cuant
found in bag: distanc
found in bag: hay
found in bag: ent
Confianza del resultado >>>>>> 0.9997384
 El ambito de la conversacio´n es ;  cuantadistancia
You: Cuanto tardas en llagar a casa
found in bag: cuanto
found in bag: tarda
found in bag: en
found in bag: a
found in bag: cas
Confianza del resultado >>>>>> 0.9999

In [0]:
# save model to file

#pickle.dump(model, open("katana-assistant-model.pkl", "wb"))
#https://www.tensorflow.org/guide/keras/save_and_serialize

model.save('model_keras.h5') # #model_keras /model_preguntas
#new_model = model.load_model('path_to_my_model.h5')
from tensorflow import keras

mi_modelo = keras.models.load_model('model_keras.h5') #model_keras /model_preguntas

In [0]:
# save all of our data structures
pickle.dump( {'words':words, 'classes':classes, 'train_x':train_x, 'train_y':train_y}, open( "datos_modelo.pkl", "wb" ) ) #datos_modelo #info_modelo_preguntas

In [0]:
data = pickle.load( open( "katana-assistant-data.pkl", "rb" ) )
words = data['words']
classes = data['classes']

In [0]:
inputvar = pd.DataFrame([p], dtype=float, index=['input'])
results = model.predict([bag_of_words(inp, words)])
results_index = numpy.argmax(results)
tag = labels[results_index]

for tg in data["intents"]:
    if tg['tag'] == tag:
        responses = tg['responses']

        print(random.choice(responses))

In [0]:
def classify_local(sentence):
    ERROR_THRESHOLD = 0.25
    
    # generate probabilities from the model
    input_data = pd.DataFrame([bow(sentence, words)], dtype=float, index=['input'])
    results = mi_modelo.predict([input_data])[0]
    # filter out predictions below a threshold, and provide intent index
    results = [[i,r] for i,r in enumerate(results) if r>ERROR_THRESHOLD]
    # sort by strength of probability
    results.sort(key=lambda x: x[1], reverse=True)
    return_list = []
    for r in results:
        return_list.append((classes[r[0]], str(r[1])))
    # return tuple of intent and probability
    
    return return_list

In [0]:
classify_local('En que parte de Madrid vives')
#You are sooon boring
#classify_local('Hello, good day!')

In [0]:
##/*********************************************************
# C H A T T E R B O T 

In [0]:
!python -m chatterbot --version

from chatterbot import ChatBot
from chatterbot.trainers import ListTrainer
#print(chatterbot.__version__)
#!python chatterbot.show
    

1.0.5


In [0]:
import os
import spacy
from google.colab import drive

#from chatterbot.trainers import ListTrainer

from chatterbot.trainers import ChatterBotCorpusTrainer

bot=ChatBot('Bot')
#bot.set_trainer(ListTrainer) #ERROR 'ChatBot' object has no attribute 'set_trainer'
trainer = ChatterBotCorpusTrainer(bot)
trainer.train("chatterbot.corpus.spanish")

print("Ready")


#****************************************************************
#trainer = ListTrainer(chatbot)
#trainer.train(conversation)


# Create a new trainer for the chatbot
#trainer = ChatterBotCorpusTrainer(chatbot)

# Train the chatbot based on the english corpus
#trainer.train("chatterbot.corpus.english")

# Get a response to an input statement
#chatbot.get_response("Hello, how are you today?")
#********************************************************************




#for files in os.listdir('D:/chatterbot-corpus-master/chatterbot_corpus/data/english/'):
#with open( 'gdrive/My Drive/data/twitter/metadata.pkl', 'rb') as f:
    #data= open('D:/chatterbot-corpus-master/chatterbot_corpus/data/english/'+files,'r').readlines()
for files in os.listdir('drive/My Drive/data/spanish'):
    print("files es : " , files)
    data= open('drive/My Drive/data/spanish/' + files,'r').readlines()
    #bot.train(data)
    trainer.train('drive/My Drive/data/spanish/' + files)
    #trainer.train("data/trainingdata.yml")

while True:
    message=input('You: ')
    if message.strip() != 'Bye':
        reply=bot.get_response(message)
        #confi=bot.get_response(confidence)
        #print("confi:", confi)
        print('ChatBot:',reply)
    if message.strip() == 'Bye':
        print('ChatBot : Bye')
        break

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
Training conversations.yml: [####################] 100%
Training greetings.yml: [####################] 100%

llego
files es :  conversations.yml
Training conversations.yml: [####################] 100%
files es :  emociones.yml
files es :  IA.yml
Training IA.yml: [####################] 100%
files es :  perfilbot

files es :  trivia.yml

files es :  greetings.yml
Training greetings.yml: [####################] 100%
files es :  psicologia.yml

files es :  dinero.yml
Training dinero.yml: [####################] 100%


KeyboardInterrupt: ignored

In [0]:
import spacy

import es_core_news_sm
nlp = es_core_news_sm.load()
print(" Version de Spacy es ", spacy.__version__)
print("1mera prueva")

doc = nlp(u'Facebook esta trabajando hasta el 15 de Abril')

for token in doc:
    print(token.text, token.lemma_, token.pos_, token.tag_, token.dep_,
            token.shape_, token.is_alpha, token.is_stop)
while True:
    message=input('You: ')
    if message.strip() != 'Bye':
        reply=bot.get_response(message)
        print('ChatBot:',reply)
    if message.strip() == 'Bye':
        print('ChatBot : Bye')
        break

AttributeError: ignored